# Install libraries

In [1]:
# !pip3 install polars
# !pip3 install Web3

# Import libraries

In [2]:
from web3 import Web3
import polars as pl
import json

# Functions

In [3]:
def create_connection(node_ip="http://127.0.0.1:8545"):
    # Connect to the local Ethereum node
    w3 = Web3(Web3.HTTPProvider(node_ip))

    # Check connection
    if not w3.is_connected():
        raise Exception("Failed to connect to the Ethereum node.")

    # Use the first account for transactions (ensure it's unlocked)
    account = w3.eth.accounts[0]
    
    return account, w3

In [4]:
def read_data(data_path='data.parquet'):
    """
    Load data from a Parquet file using Polars.
    
    Args:
        data_path (str): Path to the Parquet file.
    
    Returns:
        list[dict]: Data loaded from the Parquet file, converted to a list of dictionaries.
    """
    # Load data from a Parquet file into a Polars DataFrame
    df = pl.read_parquet(data_path)
    
    # Convert Polars DataFrame to a list of dictionaries
    return df.to_dicts()

In [5]:
# Function to store data in blockchain
def store_data_in_blockchain(data_to_store, account, w3):
    # Convert the data to hexadecimal (Ethereum stores data in hex)
    data_hex = Web3.to_hex(text=json.dumps(data_to_store))

    # Create and send a transaction
    tx = {
        'from': account,
        'to': None,  # Null address for data-only transaction
        'value': 0,  # No Ether transfer
        'gas': 3000000,  # Gas limit
        'gasPrice': w3.to_wei('20', 'gwei'),
        'data': data_hex
    }
    tx_hash = w3.eth.send_transaction(tx)
    return tx_hash

In [6]:
# Function to retrieve data from blockchain
def retrieve_data_from_blockchain(w3, tx_hash):
    # Get the transaction details
    tx = w3.eth.get_transaction(tx_hash)
    # Decode the `input` field from HexBytes to text
    return Web3.to_text(hexstr=tx['input'].hex())

In [7]:
# Store data
def store_data(data, account, w3, tx_hashes):
    for record in data:
        tx_hash = store_data_in_blockchain(record, account, w3)
        #print(f"Stored record with transaction hash: {tx_hash.hex()}")
        tx_hashes.append(tx_hash)
    return tx_hashes

In [8]:
# Retrieve data
def retrive_data(w3, tx_hashes):
    for tx_hash in tx_hashes:
        stored_data = retrieve_data_from_blockchain(w3, tx_hash)
        print(f"Retrieved data from transaction {tx_hash.hex()}: {stored_data}")

In [9]:
# Function to append new data to the blockchain
def append_data_to_blockchain(record, Web3, account, w3):
    """
    Add a new record (or update) to the blockchain.
    
    Args:
        record (dict): The record to store or update.
    
    Returns:
        str: Transaction hash of the appended record.
    """
    # Convert the data to hexadecimal (Ethereum stores data in hex)
    data_hex = Web3.to_hex(text=json.dumps(record))

    # Create and send a transaction
    tx = {
        'from': account,
        'to': None,  # Null address for data-only transaction
        'value': 0,  # No Ether transfer
        'gas': 3000000,  # Gas limit
        'gasPrice': w3.to_wei('20', 'gwei'),
        'data': data_hex
    }
    tx_hash = w3.eth.send_transaction(tx)
    return tx_hash

In [10]:
# Function to fetch and decode the latest transaction for a specific key
def get_latest_record(key, w3, key_field='vin'):
    """
    Fetch the latest record for a given key from the blockchain.

    Args:
        key (str): The unique identifier for the record.
        key_field (str): The field name used as the key in the record.

    Returns:
        dict or None: The latest record if found, otherwise None.
    """
    latest_record = None

    # Iterate through the blockchain's transactions
    for block_number in range(w3.eth.block_number + 1):
        block = w3.eth.get_block(block_number, full_transactions=True)
        for tx in block.transactions:
            if tx.to is None:  # Data-only transactions
                # Decode the transaction input (data field)
                data_hex = tx.input.hex()  # Convert HexBytes to a hex string
                try:
                    record = json.loads(Web3.to_text(hexstr=data_hex))
                    if record.get(key_field) == key:
                        latest_record = record
                except Exception as e:
                    print(f"Error decoding transaction: {e}")

    return latest_record

In [11]:
def get_record_history(w3, key, key_field='vin'):
    """
    Fetch the history of a specific record from the blockchain.

    Args:
        key (str): The unique identifier for the record.
        key_field (str): The field name used as the key in the record.

    Returns:
        list[dict]: A list of all records matching the key in their history.
    """
    history = []

    # Iterate through all blocks
    for block_number in range(w3.eth.block_number + 1):
        block = w3.eth.get_block(block_number, full_transactions=True)
        for tx in block.transactions:
            if tx.to is None:  # Data-only transactions
                try:
                    # Decode the transaction input (data field)
                    data_hex = tx.input.hex()  # Convert HexBytes to a hex string
                    record = json.loads(Web3.to_text(hexstr=data_hex))
                    
                    # Check if the transaction contains the relevant key
                    if record.get(key_field) == key:
                        history.append(record)
                except Exception as e:
                    print(f"Error decoding transaction: {e}")

    return history

# Main

## Create connection

In [12]:
tx_hashes = []

account, w3 = create_connection()

## Read data from parquet

In [13]:
data = read_data("../Data/Transform/Small/data.parquet")

## Save data to blockchain

In [14]:
tx_hashes = store_data(data, account, w3, tx_hashes)

## Retrive data from blockchain

In [15]:
retrive_data(w3, tx_hashes)

Retrieved data from transaction b77d39e4c836a4c590b6e23c623eb9350a5b79a15f3b8a1ea5d248e0cb7f2a90: {"vin": "82HFE9767U326DEZ2", "license_plate": "WU37 WRN", "vehicle_make": "Mitsubishi", "vehicle_model": "Montero", "vehicle_year": 1999, "full_vehicleInfo": {"Year": 1999, "Make": "Mitsubishi", "Model": "Montero", "Category": "SUV"}, "vehicle_category": "SUV", "vehicle_make_model": "Mitsubishi Montero", "vehicle_year_make_model": "1999 Mitsubishi Montero", "vehicle_year_make_model_cat": "1999 Mitsubishi Montero (SUV)"}
Retrieved data from transaction 6f76e0e63b29f72413fad000d70a5bf32a7eee56ba501add9c9c4bff62cc27f7: {"vin": "H1AUMH0D9M76R7NNG", "license_plate": "XO18 RDM", "vehicle_make": "Ferrari", "vehicle_model": "612 Scaglietti", "vehicle_year": 2006, "full_vehicleInfo": {"Year": 2006, "Make": "Ferrari", "Model": "612 Scaglietti", "Category": "Coupe"}, "vehicle_category": "Coupe", "vehicle_make_model": "Ferrari 612 Scaglietti", "vehicle_year_make_model": "2006 Ferrari 612 Scaglietti", 

# Testing

## Test 1 - fetch data from blockchain

In [16]:
# Retrieve the latest record for a specific VIN
vin = "82HFE9767U326DEZ2"
latest_record = get_latest_record(vin, w3)
print(f"Latest record for VIN {vin}: {latest_record}")

Latest record for VIN 82HFE9767U326DEZ2: {'vin': '82HFE9767U326DEZ2', 'license_plate': 'WU37 WRN', 'vehicle_make': 'Mitsubishi', 'vehicle_model': 'Montero', 'vehicle_year': 1999, 'full_vehicleInfo': {'Year': 1999, 'Make': 'Mitsubishi', 'Model': 'Montero', 'Category': 'SUV'}, 'vehicle_category': 'SUV', 'vehicle_make_model': 'Mitsubishi Montero', 'vehicle_year_make_model': '1999 Mitsubishi Montero', 'vehicle_year_make_model_cat': '1999 Mitsubishi Montero (SUV)'}


In [17]:
# Retrieve the latest record for a specific VIN
vin = "H1AUMH0D9M76R7NNG"
latest_record = get_latest_record(vin, w3)
print(f"Latest record for VIN {vin}: {latest_record}")

Latest record for VIN H1AUMH0D9M76R7NNG: {'vin': 'H1AUMH0D9M76R7NNG', 'license_plate': 'XO18 RDM', 'vehicle_make': 'Ferrari', 'vehicle_model': '612 Scaglietti', 'vehicle_year': 2006, 'full_vehicleInfo': {'Year': 2006, 'Make': 'Ferrari', 'Model': '612 Scaglietti', 'Category': 'Coupe'}, 'vehicle_category': 'Coupe', 'vehicle_make_model': 'Ferrari 612 Scaglietti', 'vehicle_year_make_model': '2006 Ferrari 612 Scaglietti', 'vehicle_year_make_model_cat': '2006 Ferrari 612 Scaglietti (Coupe)'}


In [18]:
# Retrieve the latest record for a specific VIN
vin = "JPC53EJ63E7RHWPAP"
latest_record = get_latest_record(vin, w3)
print(f"Latest record for VIN {vin}: {latest_record}")

Latest record for VIN JPC53EJ63E7RHWPAP: {'vin': 'JPC53EJ63E7RHWPAP', 'license_plate': 'GV19IWV', 'vehicle_make': 'GMC', 'vehicle_model': 'Envoy', 'vehicle_year': 2002, 'full_vehicleInfo': {'Year': 2002, 'Make': 'GMC', 'Model': 'Envoy', 'Category': 'SUV'}, 'vehicle_category': 'SUV', 'vehicle_make_model': 'GMC Envoy', 'vehicle_year_make_model': '2002 GMC Envoy', 'vehicle_year_make_model_cat': '2002 GMC Envoy (SUV)'}


In [19]:
# Retrieve the latest record for a specific VIN
license_plate = "GV19IWV"
latest_record = get_latest_record(license_plate, w3, "license_plate")
print(f"Latest record for license plate {license_plate}: {latest_record}")

Latest record for license plate GV19IWV: {'vin': 'JPC53EJ63E7RHWPAP', 'license_plate': 'GV19IWV', 'vehicle_make': 'GMC', 'vehicle_model': 'Envoy', 'vehicle_year': 2002, 'full_vehicleInfo': {'Year': 2002, 'Make': 'GMC', 'Model': 'Envoy', 'Category': 'SUV'}, 'vehicle_category': 'SUV', 'vehicle_make_model': 'GMC Envoy', 'vehicle_year_make_model': '2002 GMC Envoy', 'vehicle_year_make_model_cat': '2002 GMC Envoy (SUV)'}


## Test 2 - update the record and get the updated version

In [20]:
# Example Usage
# Append updated record to blockchain
updated_record = {
    "vin": "82HFE9767U326DEZ2",
    "license_plate": "WU37 WRN",
    "vehicle_make": "Mitsubishi",
    "vehicle_model": "Outlander",
    "vehicle_year": 2000,
    "full_vehicleInfo": {
        "Year": 2000,
        "Make": "Mitsubishi",
        "Model": "Outlander",
        "Category": "SUV"
    },
    "vehicle_category": "SUV",
    "vehicle_make_model": "Mitsubishi Outlander",
    "vehicle_year_make_model": "2000 Mitsubishi Outlander",
    "vehicle_year_make_model_cat": "2000 Mitsubishi Outlander (SUV)"
}

tx_hash = append_data_to_blockchain(updated_record, Web3, account, w3)
print(f"Updated record added to blockchain with transaction hash: {tx_hash.hex()}")

Updated record added to blockchain with transaction hash: c2e973d4959f1945ad27e254b3d46bed7bf4c09fb4ffb739eadbc331d8b16407


In [21]:
# Retrieve the latest record for a specific VIN
vin = "82HFE9767U326DEZ2"
latest_record = get_latest_record(vin, w3)
print(f"Latest record for VIN {vin}: {latest_record}")

Latest record for VIN 82HFE9767U326DEZ2: {'vin': '82HFE9767U326DEZ2', 'license_plate': 'WU37 WRN', 'vehicle_make': 'Mitsubishi', 'vehicle_model': 'Outlander', 'vehicle_year': 2000, 'full_vehicleInfo': {'Year': 2000, 'Make': 'Mitsubishi', 'Model': 'Outlander', 'Category': 'SUV'}, 'vehicle_category': 'SUV', 'vehicle_make_model': 'Mitsubishi Outlander', 'vehicle_year_make_model': '2000 Mitsubishi Outlander', 'vehicle_year_make_model_cat': '2000 Mitsubishi Outlander (SUV)'}


## Test 3 - Get historical data to see updates

In [22]:
vin = "82HFE9767U326DEZ2"
get_record_history(w3, vin)

[{'vin': '82HFE9767U326DEZ2',
  'license_plate': 'WU37 WRN',
  'vehicle_make': 'Mitsubishi',
  'vehicle_model': 'Montero',
  'vehicle_year': 1999,
  'full_vehicleInfo': {'Year': 1999,
   'Make': 'Mitsubishi',
   'Model': 'Montero',
   'Category': 'SUV'},
  'vehicle_category': 'SUV',
  'vehicle_make_model': 'Mitsubishi Montero',
  'vehicle_year_make_model': '1999 Mitsubishi Montero',
  'vehicle_year_make_model_cat': '1999 Mitsubishi Montero (SUV)'},
 {'vin': '82HFE9767U326DEZ2',
  'license_plate': 'WU37 WRN',
  'vehicle_make': 'Mitsubishi',
  'vehicle_model': 'Outlander',
  'vehicle_year': 2000,
  'full_vehicleInfo': {'Year': 2000,
   'Make': 'Mitsubishi',
   'Model': 'Outlander',
   'Category': 'SUV'},
  'vehicle_category': 'SUV',
  'vehicle_make_model': 'Mitsubishi Outlander',
  'vehicle_year_make_model': '2000 Mitsubishi Outlander',
  'vehicle_year_make_model_cat': '2000 Mitsubishi Outlander (SUV)'}]

# Other